# File IO

> Saving and loading data.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#|default_exp fileio
#|export
import pathlib
import yaml
import json
import pickle
from pathlib import Path
from os import walk
import numpy as np
import csv
from nbx.utils import listmap, Bunch
Path = pathlib.Path

In [ ]:
#|export
def ls_bash(path="./", absolute=False, posix=False):
    path = Path(path)
    _f = []
    _d = []
    for (dirpath, dirnames, filenames) in walk(path):
        _f.extend(filenames)
        _d.extend(dirnames)
        break

    f = _f
    d = _d
    
    if posix:
        f = listmap(Path, _f)
        d = listmap(Path, _d)    

    return Bunch(path=path, files=sorted(f), dirs=sorted(d), folders=sorted(d))

ls = ls_bash

In [ ]:
ls_bash("./")

{'path': PosixPath('.'),
 'files': ['.DS_Store',
  '.gitattributes',
  '00_core.ipynb',
  '01_utils.ipynb',
  '02_fileio.ipynb',
  '03_templ.ipynb',
  '04_confspace.ipynb',
  '05_nbparser.ipynb',
  '_01_logging.ipynb',
  '_04_nbparser - original.ipynb',
  '_04a_parser.ipynb',
  '_04b_nbparser.ipynb',
  'index.ipynb'],
 'dirs': ['.ipynb_checkpoints', 'nbx_example'],
 'folders': ['.ipynb_checkpoints', 'nbx_example']}

In [ ]:
#|export
def save(obj, fname):
    """
    Saves an object to a file.
    
    Supported formats are `.yaml`, `.json`, `.npy`, `.npz`
    and everything else will be "pickled".
    """

    p = Path(fname)
    name   = p.stem
    format = p.suffix

    if format==".yaml":
        with open(p, 'w') as f:
            yaml.dump(obj, f,
                allow_unicode=True,
                sort_keys=False,
                indent=4,
                explicit_start=True)

    elif format==".json":
        with open(p, 'w') as f:
            json.dump(obj, f,
                        indent=4)

    elif format==".npy" or format==".npz":
        np.save(p, obj)

    else:
        with open(p, 'wb') as f:
            pickle.dump(obj, f)

In [ ]:
#|export
def dump(obj, fname): 
    """Dumps an object to file.""" 
    return save(obj, fname)

In [ ]:
#|export
def load(fname, format=None):
    
    p = Path(fname)
    name = p.stem
    
    if format is None:
        format = p.suffix
            
    if format==".yaml":
        with open(p, 'r') as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    elif format==".json":
        with open(p, 'r') as f:
            return json.load(f)

    elif format==".npy" or format==".npz":
        return np.load(p)

    elif format==".txt":
        with open(p, 'r') as f:
            lines = f.read().split('\n')
            return lines
    
    elif format==".pkl":
        with open(p, 'rb') as f:
            return pickle.load(f)
        
    elif format==".tsv":
        with open(p) as f:
            X = csv.reader(f, delimiter="\t", quotechar='"')
            return [x for x in X]
    
    elif format==".csv":
        with open(p) as f:
            X = csv.reader(f, delimiter=",", quotechar='"')            
            return [x for x in X]
                
    else:
        supported = [".yaml", ".json", ".npy", ".npz", ".txt", ".pkl", ".tsv", ".csv"]
        raise ValueError(f"Document type not supported: should be one of the following: \n\t{supported}")
        
    

In [ ]:
arr = np.random.rand(10)
dump(arr, "_temp/numpy.npy")
assert np.any(load("_temp/numpy.npy") == arr, None)
!rm "_temp/numpy.npy"

In [ ]:
#|export
def find_file_upwards(fname):
    path = Path.cwd()
    while path != path.parent and not (path/fname).exists(): path = path.parent
    fname = path/fname
    return fname